In [3]:
import re 
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder

from collections import defaultdict

from keras.preprocessing.text import text_to_word_sequence

import nltk

/home/a416253190/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [4]:
cloth = pd.read_csv("/mnt/disks/~/clean.csv")
print(cloth.shape)
cloth.head(3)

(758065, 13)


,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat1,cat2,cat3,cat4,cat5,target
0,mlb cincinnati reds t shirt size xl,3,Men/Tops/T-shirts,4786,10.0,1,no description yet,554,859,827,950,950,-0.369464
1,ava-viv blouse,1,Women/Tops & Blouses/Blouse,4180,10.0,1,adorable top with a hint of lace and a key hol...,934,860,104,950,950,-0.369464
2,24k gold plated rose,1,Women/Jewelry/Necklaces,4786,44.0,0,complete with certificate of authenticity,934,480,584,950,950,0.000978


# Tokenize
Tokenize the text and add 2 columns of the tokenized words

In [5]:
cloth['token_name'] = [text_to_word_sequence(w) for w in cloth['name']]
cloth['token_item_description'] = [text_to_word_sequence(w) for w in cloth['item_description']]

# What's the item?

In [6]:
items = ['missing', 'shirt', 't-shirt', 't shirt', 'pants', \
         'jeans', 'trousers', 'jacket', 'coat', \
         'sweater', 'hat', 'cap', 'dress', 'shorts', \
         'underwear', 'socks', 'blouse', 'shoes', 'boots']

def getItem(text):
    item = 'missing'
    for w in items:
        if re.search(w, text):
            item = w
    return item

cloth['item_name'] = [getItem(text) for text in cloth['name']]

item_name_le = LabelEncoder()
item_name_le.fit(items)
cloth.item_name = item_name_le.transform(cloth.item_name)
# save the model to disk
item_name_le_file = '/mnt/disks/~/item_name_le.sav'
pickle.dump(item_name_le, open(item_name_le_file, 'wb'), protocol=2)

# Color

In [7]:
colors = ['missing', 'white', 'black', 'pink', 'red', 'green', \
          'blue', 'yellow', 'brown', 'purple', 'violet', 'dark']

def getColor(text):
    color = 'missing'
    for w in colors:
        if re.search(w, text):
            color = w
    return color

cloth['color'] = [getColor(text) for text in cloth['item_description']]

# Color
color_le = LabelEncoder()
color_le.fit(colors)
cloth.color = color_le.transform(cloth.color)
# save the model to disk
color_le_file = '/mnt/disks/~/color_le.sav'
pickle.dump(color_le, open(color_le_file, 'wb'), protocol=2)

# New or not?
0 stands for old. 1 stands for new.

In [8]:
def isNew(text):
    match1 = re.search('new', text)
    match2 = re.search('never worn', text)
    if match1:
        flag = 1
    elif match2:
        flag = 1
    else: flag = 0
    return flag

cloth['isnew'] = [isNew(t) for t in cloth['item_description']]

# Style of the cloth
Such as classic, casual, old-fashioned, vintage, retro, sport and formal.

In [9]:
styles = ['missing', 'active','adjustable','affordable','asymmetrical',\
          'beautiful','casual','classic','comfortable',\
          'contemporary','cool','custom','cute','discount',\
          'easy to maintain','eco friendly','elegant','engineered',\
          'environmentally friendly','exciting','exposed','flexible',\
          'formal','half priced','high-waisted','knee length',\
          'knit','layered','lightweight','metallic','modern',\
          'moisture-wicking','old-fashioned','on trend','organic',\
          'oversized','patchwork','patterned','peasant','polished',\
          'pretty','professional','protective','retro',\
          'rocky','romantic','scooped-neck','semi-formal','sexy',\
          'simple','soft','solid','sport','standard','superior',\
          'supersoft','sweetheart','symmetrical','tailored',\
          'textured','trend','unique','vintage','voluminous',
          'water-resistant','waterproof','wear-anywhere','whimsical']

def getStyle(text):
    style = 'missing'
    for w in styles:
        if re.search(w, text):
            style = w
    return style

cloth['style'] = [getStyle(text) for text in cloth['item_description']]

style_le = LabelEncoder()
style_le.fit(styles)
cloth['style'] = style_le.transform(cloth['style'])
# save the model to disk
style_le_file = '/mnt/disks/~/style_le.sav'
pickle.dump(style_le, open(style_le_file, 'wb'), protocol=2)

# Aging of the style
Whether it's 60s, 70s, 80s, 90s, modern or missing.

In [10]:
ages = ['missing', '60s', '70s', '80s', '90s', 'modern']

def age(text):
    flag = 'missing'
    match60 = re.search('60s', text)
    match70 = re.search('70s', text)
    match80 = re.search('80s', text)
    match90 = re.search('90s', text)
    match00 = re.search('modern', text)

    if match60: flag = '60s'
    if match70: flag = '70s'
    if match80: flag = '80s'
    if match90: flag = '90s'
    if match00: flag = 'modern'
    return flag
cloth['age'] = [age(t) for t in cloth['item_description']]

age_le = LabelEncoder()
age_le.fit(ages)
cloth.age = age_le.transform(cloth.age)
# save the model to disk
age_le_file = '/mnt/disks/~/age_le.sav'
pickle.dump(age_le, open(age_le_file, 'wb'), protocol=2)

# Wash style
0 stands for missing. 1 stands for hand wash. 2 stands for machine washable.

In [11]:
def getWash(text):
    flag = 0
    match1 = re.search('hand wash', text)
    match2 = re.search('machine wash', text)
    if match1:
        flag = 1
    if match2:
        flag = 2
    return flag

cloth['wash_style'] = [getWash(text) for text in cloth['item_description']]

# Fit

In [12]:
fits = ['missing', 'loose','standard-fit','fitted','tight','skinny','skinny-fit']

def getFit(text):
    fit = 'missing'
    for w in fits:
        if re.search(w, text):
            fit = w       
    return fit

cloth['fit'] = [getFit(text) for text in cloth['item_description']]

fit_le = LabelEncoder()
fit_le.fit(fits)
cloth.fit = fit_le.transform(cloth.fit)
# save the model to disk
fit_le_file = '/mnt/disks/~/fit_le.sav'
pickle.dump(fit_le, open(fit_le_file, 'wb'), protocol=2)

# Size
0 stands for missing. 1 stands for xs. 2 stands for s.
3 stands for m. 4 stands for l. 5 stands for xl. 6 stands for xxl.

In [13]:
sizes = ['missing', 'xs','s','m','l','xl','xxl','extra small','small','medium','large','extra large']

def getSize(text):
    size = 0
    for w in sizes:
        if re.search(w, text):
            size = w  
    if (size == 'xs') | (size == 'extra small'):
        size = 1
    elif (size == 's') | (size == 'small'):
        size = 2
    elif (size == 'm') | (size == 'medium'):
        size = 3
    elif (size == 'l') | (size == 'large'):
        size = 4
    elif (size == 'xl') | (size == 'extra large'):
        size = 5
    elif size == 'xxl':
        size = 6
    else: size = 0
    return size

cloth['size'] = [getSize(text) for text in cloth['item_description']]

# Fabric

In [14]:
fabrics = ['missing', 'canvas','chambray','cotton','leather','flax',\
           'full-grain leather','jacquard','jean','jersey',\
           'lace','mesh','moleskin','nylon','organza','oxford',\
           'polyester','silk','top-grain leather']

def getFabric(text):
    fabric = 'missing'
    for w in fabrics:
        if re.search(w, text):
            fabric = w
    return fabric

cloth['fabric'] = [getFabric(text) for text in cloth['item_description']]

fabric_le = LabelEncoder()
fabric_le.fit(fabrics)
cloth.fabric = fabric_le.transform(cloth.fabric)
# save the model to disk
fabric_le_file = '/mnt/disks/~/fabric_le.sav'
pickle.dump(fabric_le, open(fabric_le_file, 'wb'), protocol=2)

# Quality of the cloth

In [15]:
qualities = ['missing', 'poor','premium','high-quality','low-quality','medium-quality','normal-quality']

def getQuality(text):
    quality = 'missing'
    for w in qualities:
        if re.search(w, text):
            quality = w
    return quality

cloth['quality'] = [getQuality(text) for text in cloth['item_description']]

quality_le = LabelEncoder()
quality_le.fit(qualities)
cloth.quality = quality_le.transform(cloth.quality)
# save the model to disk
quality_le_file = '/mnt/disks/~/quality_le.sav'
pickle.dump(quality_le, open(quality_le_file, 'wb'), protocol=2)

# Sentiment Extraction
Since we are not focusing on the sentiment analysis part, we will just cite a pre-clothed model called vaderSentiment to get the result of Positive, Neutral and Negtive.

In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
classifier = SentimentIntensityAnalyzer()

def getSentiment(text):
    vs = classifier.polarity_scores(text)
    if vs['compound'] >= 0.5:
        sentiment = 1
    elif vs['compound'] <= -0.5:
        sentiment = -1
    else:
        sentiment = 0
    return sentiment

cloth['sentiment'] = [getSentiment(text) for text in cloth['item_description']]

In [17]:
pd.set_option('display.max_columns', None)
print(cloth.shape)
cloth.head()

(758065, 26)


,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat1,cat2,cat3,cat4,cat5,target,token_name,token_item_description,item_name,color,isnew,style,age,wash_style,fit,size,fabric,quality,sentiment
0,mlb cincinnati reds t shirt size xl,3,Men/Tops/T-shirts,4786,10.0,1,no description yet,554,859,827,950,950,-0.369464,"[mlb, cincinnati, reds, t, shirt, size, xl]","[no, description, yet]",15,5,0,27,4,0,2,2,11,2,0
1,ava-viv blouse,1,Women/Tops & Blouses/Blouse,4180,10.0,1,adorable top with a hint of lace and a key hol...,934,860,104,950,950,-0.369464,"[ava, viv, blouse]","[adorable, top, with, a, hint, of, lace, and, ...",0,6,0,27,4,0,2,4,8,2,1
2,24k gold plated rose,1,Women/Jewelry/Necklaces,4786,44.0,0,complete with certificate of authenticity,934,480,584,950,950,0.000978,"[24k, gold, plated, rose]","[complete, with, certificate, of, authenticity]",8,5,0,27,4,0,2,4,11,2,0
3,bundled items requested for ruie,3,Women/Other/Other,4786,59.0,0,"banana republic bottoms, candies skirt with ma...",934,609,609,950,950,0.076625,"[bundled, items, requested, for, ruie]","[banana, republic, bottoms, candies, skirt, wi...",8,5,0,27,4,0,2,4,11,2,0
4,acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,79,64.0,0,size small but straps slightly shortened to fi...,934,824,894,950,950,0.097672,"[acacia, pacific, tides, santorini, top]","[size, small, but, straps, slightly, shortened...",8,5,0,27,4,0,2,2,11,2,1


In [18]:
cloth.to_csv("/mnt/disks/~/features.csv",index=None)

# Ridge Regression

In [20]:
X = cloth.loc[:,['item_name','item_condition_id','brand_name','shipping','cat1','cat2','cat3',\
                 'color','isnew','style','age','wash_style','fit','size','fabric','quality','sentiment']]
y = cloth['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [21]:
from sklearn.linear_model import Ridge

def rmsle(ytrue,y):
    return np.sqrt(mean_squared_log_error(ytrue,y))

rr = Ridge(solver="sag", fit_intercept=True, random_state=205, alpha=3)
rr.fit(X_train,y_train)

y_pred_tr = rr.predict(X_train)
score_train = rmsle(y_train,y_pred_tr)

y_pred = rr.predict(X_test)
score_test = rmsle(y_test,y_pred)

print(score_train)
print(score_test)

0.773994796535
0.77446229171


# Random Forest

In [22]:
from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestRegressor(n_estimators=50,min_samples_leaf=50,max_depth=20)
rfc.fit(X_train,y_train)

y_pred_tr = rfc.predict(X_train)
score_train = rmsle(y_train,y_pred_tr)

y_pred = rfc.predict(X_test)
score_test = rmsle(y_test,y_pred)

print(score_train)
print(score_test)
print(rfc.feature_importances_)

0.555856693112
0.563924118939
[  2.07503527e-02   1.01264923e-01   4.87601866e-01   1.25004750e-02
   1.00432754e-02   1.99378396e-01   1.02163753e-01   1.47350668e-02
   1.08940646e-02   9.97095673e-03   3.58175607e-07   8.97165941e-06
   5.00372195e-05   6.56871747e-03   9.71752465e-03   0.00000000e+00
   1.43512622e-02]
